# M2 Force balance system test through a bump sequence - axial actuators

This Jupyter notebook is to run the bump test of M2 to verify that the M2 force balance system works as expected in closed-loop.
Three axial actuators are issued with a fixed force of 100 N with pull/push (+/-) movement one by one.

## Import Modules

In [1]:
import asyncio
from datetime import datetime
from lsst.ts import salobj
from lsst.ts.m2com import NUM_ACTUATOR, NUM_TANGENT_LINK

## Declaration of User-defined Functions

In [2]:
async def run_bump_test(csc, force, sleep_time=5.0):
    """Run the bump test.

    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    force : float
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    # Print the information of inclinometer
    zenithAngle = csc.tel_zenithAngle.get()
    print(f"Measured zenith angle: {zenithAngle.measured:0.2f} degree.")
    print(f"Raw inclinometer angle: {zenithAngle.inclinometerRaw:0.2f} degree.")
    print(f"Processed inclinometer angle: {zenithAngle.inclinometerProcessed:0.2f} degree.")

    # select 3 actuators ring C
    for idx in range(33,54,10):
        print(f"Do the bump test of actuator: {idx}.")
        await bump_axial_actuator(csc, idx, force, sleep_time=sleep_time)

In [3]:
async def bump_axial_actuator(csc, idx_actuator, force, sleep_time=5):
    """Bump the axial actuator.
    
    Parameters
    ----------
    csc : lsst.ts.salobj.remote.Remote
        Remote object of the M2 CSC.
    idx_actuator : int
        Index of actuator.
    force : float
        Force to apply (Newton).
    sleep_time : float, optional
        Sleep time. (the default is 5.0)
    """

    # Do the positive direction first
    num_axial_actuator = NUM_ACTUATOR - NUM_TANGENT_LINK
    forces = [0] * num_axial_actuator
    forces[idx_actuator] = abs(force)

    print(f"Apply the force: {abs(force)} N.")
    await csc.cmd_applyForces.set_start(axial=forces)
    await asyncio.sleep(sleep_time)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(sleep_time)

    # Do the Negative direction
    forces[idx_actuator] = -abs(force)

    print(f"Apply the force: {-abs(force)} N.")
    await csc.cmd_applyForces.set_start(axial=forces)
    await asyncio.sleep(sleep_time)

    # Put back to origin
    print("Reset the force.")
    await csc.cmd_resetForceOffsets.set_start()
    await asyncio.sleep(sleep_time)

## Prepare the M2 CSC and put to Enabled state

In [4]:
domain = salobj.Domain()
m2 = salobj.Remote(domain, "MTM2")
await m2.start_task
await m2.cmd_setLogLevel.set_start(level=10)

In [ ]:
state_m2 = m2.evt_summaryState.get()
if state_m2.summaryState != salobj.State.ENABLED:
    await salobj.set_summary_state(m2, salobj.State.ENABLED, timeout=10)

## Do the Bump Test

In [6]:
time_start = datetime.now()
print(f"UTC time to is {time_start} now.")

force = 100
await run_bump_test(m2, force, sleep_time=30)

time_end = datetime.now()
print(f"UTC time to is {time_end} now.")

UTC time to is 2023-07-06 02:32:33.652147 now.
Measured zenith angle: 359.95 degree.
Raw inclinometer angle: 89.01 degree.
Processed inclinometer angle: -269.95 degree.
Do the bump test of actuator: 33.
Apply the force: 100 N.
Reset the force.
Apply the force: -100 N.
Reset the force.
Do the bump test of actuator: 43.
Apply the force: 100 N.
Reset the force.
Apply the force: -100 N.
Reset the force.
Do the bump test of actuator: 53.
Apply the force: 100 N.
Reset the force.
Apply the force: -100 N.
Reset the force.
UTC time to is 2023-07-06 02:38:39.694661 now.
